# SAM

In [ ]:
import cv2
import numpy as np

# Load image
image_bgr = cv2.imread("images/frame_20250429_183115.png")
if image_bgr is None:
    raise RuntimeError("Could not load image.")

image_hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
image_display = image_bgr.copy()
clicked_points = []

# Mouse callback function
def on_mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN and len(clicked_points) < 2:
        hsv_value = image_hsv[y, x]
        clicked_points.append((x, y, hsv_value))

        # Draw a small cross
        cv2.drawMarker(image_display, (x, y), (0, 0, 255), cv2.MARKER_CROSS, 10, 2)
        cv2.imshow("Click to sample HSV", image_display)

        print(f"Clicked point {len(clicked_points)}: x={x}, y={y}, HSV={hsv_value}")

        if len(clicked_points) == 2:
            print("\nDone! Use these HSV values to tune your threshold.")
            print(f"Background HSV: {clicked_points[0][2]}")
            print(f"Foreground HSV: {clicked_points[1][2]}")
            cv2.waitKey(0)
            cv2.destroyAllWindows()

# Display the image and wait for two clicks
cv2.imshow("Click to sample HSV", image_display)
cv2.setMouseCallback("Click to sample HSV", on_mouse)
print("Click once on the BACKGROUND, then once on the CATHETER TIP.")

input_point = np.array([clicked_points[0][:2], clicked_points[1][:2]])
input_label = np.array([0, 1]) # 0 for background, 1 for foreground
print("Input point:", input_point)

cv2.waitKey(0)
cv2.destroyAllWindows()

# SAM model and image set up

In [ ]:
import cv2
import numpy as np
import torch
from segment_anything import sam_model_registry, SamPredictor
from matplotlib import pyplot as plt

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))    

# ──────────────────────────────────────────────────────────────────────────────
# 1. Load SAM ViT-B on CPU
# ──────────────────────────────────────────────────────────────────────────────
checkpoint_path = "/home/arclab/repos/segment-anything/checkpoints/sam_vit_b_01ec64.pth"
model_type = "vit_b"

sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
sam.to("cuda")
predictor = SamPredictor(sam)

# ──────────────────────────────────────────────────────────────────────────────
# 2. Load image and create coarse foreground mask (non-green areas)
# ──────────────────────────────────────────────────────────────────────────────
img_bgr = cv2.imread("images/frame_20250429_183115.png")
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
print("Original image shape:", img_bgr.shape)


lower_green = np.array([35, 30, 100])
upper_green = np.array([85, 255, 255])
green_mask = cv2.inRange(img_hsv, lower_green, upper_green)
foreground_mask = cv2.bitwise_not(green_mask)
binary_mask = np.where(foreground_mask > 0, 1, 0).astype(np.uint8)


if img_bgr is None:
    raise RuntimeError("Could not load image.")

predictor.set_image(img_rgb)


# Automated point and box prompting

In [ ]:
import cv2
import numpy as np
import torch
from segment_anything import SamPredictor, sam_model_registry
import matplotlib.pyplot as plt


# ===============================================
# ============ APPROACH 1: POINT PROMPT =========
# ===============================================
# Get the centroid of the binary mask
M = cv2.moments(binary_mask)
if M["m00"] != 0:
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])
    point_coords = np.array([[cx, cy]])
else:
    raise ValueError("No foreground pixels found in binary mask")

print("Centroid coordinates:", cx, cy)


point_labels = np.array([1])  # 1 = foreground

masks_point, scores_point, _ = predictor.predict(
    point_coords=point_coords,
    point_labels=point_labels,
    multimask_output=False,
)

# ===============================================
# ============ APPROACH 2: BOX PROMPT ===========
# ===============================================
x, y, w, h = cv2.boundingRect(binary_mask)
box = np.array([[x, y, x + w, y + h]])

masks_box, scores_box, _ = predictor.predict(
    box=box,
    multimask_output=False
)

# ---- Visualize Results ----
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

axs[0].imshow(image_rgb)
axs[0].set_title("Original Image")

axs[1].imshow(image_rgb)
axs[1].imshow(masks_point[0], alpha=0.5, cmap='Reds')
axs[1].scatter(cx, cy, c='blue', s=40)
axs[1].set_title("Point Prompt Result")

axs[2].imshow(image_rgb)
axs[2].imshow(masks_box[0], alpha=0.5, cmap='Reds')
axs[2].add_patch(plt.Rectangle((x, y), w, h, edgecolor='blue', facecolor='none', lw=2))
axs[2].set_title("Box Prompt Result")

for ax in axs:
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:

# Show original image
plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
show_points(input_point, input_label, plt.gca())
plt.title("Original Image")
plt.axis('off')
plt.show()


# Define green background HSV range
lower_green = np.array([35, 30, 100])
upper_green = np.array([85, 255, 255])
green_mask = cv2.inRange(img_hsv, lower_green, upper_green)
print("max val: ", np.max(green_mask))
print("min val: ", np.min(green_mask))

# Invert: 1 for non-green (catheter), 0 for background
foreground_mask = cv2.bitwise_not(green_mask)
print("max val: ", np.max(foreground_mask))

# Optionally clean up mask
kernel = np.ones((5,5), np.uint8)
foreground_mask = cv2.morphologyEx(foreground_mask, cv2.MORPH_OPEN, kernel)
print("max val: ", np.max(foreground_mask))
# Convert to binary mask
binary_mask = np.where(foreground_mask > 0, 1, 0).astype(np.uint8)

print("min val: ", np.min(binary_mask))
print("max val: ", np.max(binary_mask))
print("binary mask shape: ", binary_mask.shape)
# Visualize foreground mask
plt.figure(figsize=(8, 8))
plt.imshow(binary_mask, cmap='gray')
plt.title("Foreground Mask (Non-Green Areas)")
plt.axis('off')
plt.show()


In [ ]:


# # Resize the coarse mask to SAM's expected prompt input resolution
# # For ViT-B: patch size = 16 → input image (1024) / 16 = 64 → 1/4 = 16x16
# prompt_H, prompt_W = 256, 256

# coarse_mask = cv2.resize(
#     coarse_mask.astype(np.float32),
#     (prompt_W, prompt_H),
#     interpolation=cv2.INTER_NEAREST
# )

# print("Coarse mask shape:", coarse_mask.shape)
# plt.figure(figsize=(8, 8))
# plt.imshow(coarse_mask)
# plt.title("Coarse Mask (Non-Green Areas)")
# plt.axis('off')
# plt.show()

# # ──────────────────────────────────────────────────────────────────────────────
# # 3. Feed image to SAM
# # ──────────────────────────────────────────────────────────────────────────────
# image_embedding = predictor.get_image_embedding().cpu().numpy()

# print("Image embedding shape:", image_embedding.shape)
# print("Sample values:", image_embedding[0, :, :5, :5])  # Print part of it for inspection


# # # Add channel dimension → (1, 16, 16)
# # mask_input = mask_resized[np.newaxis, :, :]
# # print(f"Mask input shape: {mask_input.shape}")

# # ──────────────────────────────────────────────────────────────────────────────
# # 4. Run SAM with dense prompt only
# # ──────────────────────────────────────────────────────────────────────────────
# with torch.inference_mode():
#     masks, scores, logits = predictor.predict(
#         point_coords=input_point,
#         point_labels=input_label,
#         # box=None,
#         # mask_input=mask_input,
#         multimask_output=True
#     )

# for i, (mask, score) in enumerate(zip(masks, scores)):
#     plt.figure(figsize=(10,10))
#     plt.imshow(img_rgb)
#     show_mask(mask, plt.gca())
#     show_points(input_point, input_label, plt.gca())
#     plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
#     plt.axis('off')
#     plt.show()  

# best_mask = logits[np.argmax(scores), :, :]
# print(f"Best mask logit shape: {best_mask.shape}")

# # Run prediction again with best_mask as prompt
# refined_mask, scores, logits = predictor.predict(
#     point_coords=None,
#     point_labels=None,
#     box=None,
#     multimask_output=False,
#     mask_input=best_mask[None, :, :]
# )

# Run prediction with coarse mask as prompt
binary_mask = cv2.resize(binary_mask, (256, 256), interpolation=cv2.INTER_NEAREST)
logit_mask = np.where(binary_mask > 0, 10.0, -10.0).astype(np.float32)
print(np.max(logit_mask))
output_mask_coarse, scores, logits = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=None,
    multimask_output=False,
    mask_input=logit_mask[None, :, :]
)

print("Scores:", scores)

plt.figure(figsize=(8, 8))
plt.imshow(output_mask_coarse[0, :, :], cmap='gray')
plt.title("Coarse Mask Output")
plt.axis('off')
plt.show()

# # # Refined mask (shape: 1024x1024)
# # output_mask = masks[0,:,:].astype(np.uint8)
# # print(f"Output masks shape: {output_mask.shape}")

# # # ──────────────────────────────────────────────────────────────────────────────
# # # 5. Upsample refined mask to original image size and overlay
# # # ──────────────────────────────────────────────────────────────────────────────
# # # H_orig, W_orig = img_rgb.shape[:2]
# # # mask_fullres = cv2.resize(
# # #     refined_mask,
# # #     (W_orig, H_orig),
# # #     interpolation=cv2.INTER_NEAREST
# # # )

# # # ──────────────────────────────────────────────────────────────────────────────
# # # 6. Visualize overlay
# # # ──────────────────────────────────────────────────────────────────────────────
# plt.figure(figsize=(8, 8))
# # plt.imshow(img_rgb)
# plt.imshow(refined_mask[0, :, :], cmap='gray')
# plt.title("SAM Output Mask")
# plt.axis('off')
# plt.show()


# Automatic mask generation

In [ ]:
from segment_anything import SamAutomaticMaskGenerator

def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.array([1.0, 0.0, 0.0, 0.35])  # Red color with 35% opacity
        img[m] = color_mask
    ax.imshow(img)


# Use SAM to automatically generate masks for image
mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(img_rgb)

print("Number of masks: ", len(masks))
print(masks[0].keys())

plt.figure(figsize=(20,20))
plt.imshow(img_rgb)
show_anns(masks)
plt.axis('off')
plt.show() 

# Visualize the mask with the best predicted_iou
best_mask_ann = max(masks, key=lambda x: x['predicted_iou'])
plt.figure(figsize=(10, 10))
plt.imshow(img_rgb)
show_anns([best_mask_ann])
plt.axis('off')
plt.title("Best Mask with Highest Predicted IoU")
plt.show()

for idx, mask_ann in enumerate(masks):
    plt.figure(figsize=(10, 10))
    plt.imshow(img_rgb)
    show_anns([mask_ann])
    plt.axis('off')
    plt.title(f"Mask {idx + 1} - Predicted IoU: {mask_ann['predicted_iou']:.3f}, Stability Score: {mask_ann['stability_score']:.3f}")
    plt.show()

# Calibration

In [ ]:
import cv2
import numpy as np
import glob
import os

def calibrate_intrinsics(image_glob, pattern_size=(9,6), square_size=0.025):
    """
    Calibrate a single camera's intrinsics using checkerboard images.
    
    Args:
        image_glob (str): Glob pattern for calibration images.
        pattern_size (tuple): Number of inner corners per checkerboard row, col.
        square_size (float): Size of a checkerboard square in meters.
        
    Returns:
        ret (float): RMS re-projection error.
        K (ndarray): Camera matrix (3x3).
        dist (ndarray): Distortion coefficients.
        rvecs, tvecs: Extrinsic vectors for each image.
    """
    # Prepare object points (0,0,0), ..., scaled by square_size
    objp = np.zeros((pattern_size[0]*pattern_size[1],3), np.float32)
    objp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1,2)
    objp *= square_size

    objpoints, imgpoints = [], []
    images = glob.glob(image_glob)
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
        if not ret:
            continue
        corners2 = cv2.cornerSubPix(
            gray, corners, (11,11), (-1,-1),
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        objpoints.append(objp)
        imgpoints.append(corners2)

    ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None
    )
    print(f"Intrinsics RMS error: {ret:.4f}")
    return ret, K, dist, rvecs, tvecs

def stereo_calibrate(
    glob1, glob2, pattern_size, square_size,
    K1, d1, K2, d2,
    flags=cv2.CALIB_FIX_INTRINSIC
):
    """
    Stereo calibrate two cameras that have already been intrinsically calibrated.
    
    Args:
        glob1, glob2: Glob patterns for the two cameras' images.
        pattern_size, square_size: Checkerboard parameters.
        K1, d1, K2, d2: Intrinsics and distortions for camera 1 and 2.
        flags: calibration flags.
        
    Returns:
        R, T, E, F: Rotation, translation, essential and fundamental matrices.
    """
    objp = np.zeros((pattern_size[0]*pattern_size[1],3), np.float32)
    objp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1,2)
    objp *= square_size

    objpoints, imgpoints1, imgpoints2 = [], [], []
    imgs1 = sorted(glob.glob(glob1))
    imgs2 = sorted(glob.glob(glob2))
    for f1, f2 in zip(imgs1, imgs2):
        im1 = cv2.imread(f1); gray1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
        im2 = cv2.imread(f2); gray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
        r1, c1 = cv2.findChessboardCorners(gray1, pattern_size, None)
        r2, c2 = cv2.findChessboardCorners(gray2, pattern_size, None)
        if not (r1 and r2):
            continue
        c1 = cv2.cornerSubPix(
            gray1, c1, (11,11), (-1,-1),
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        c2 = cv2.cornerSubPix(
            gray2, c2, (11,11), (-1,-1),
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        objpoints.append(objp)
        imgpoints1.append(c1)
        imgpoints2.append(c2)

    ret, _, _, _, _, R, T, E, F = cv2.stereoCalibrate(
        objpoints, imgpoints1, imgpoints2,
        K1, d1, K2, d2,
        gray1.shape[::-1],
        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5),
        flags=flags
    )
    print(f"Stereo RMS error: {ret:.4f}")
    return R, T, E, F

def calibrate_camera_to_world(
    image_glob, world_objpoints=None,
    pattern_size=(9,6), square_size=0.025,
    K=None, dist=None
):
    """
    Solve PnP to get camera-to-world extrinsics using a known world marker.
    
    Args:
        image_glob: Glob for images where the world marker is visible.
        world_objpoints: (N×3) array of 3D points in world coords.
        pattern_size, square_size: if using a checkerboard as your world frame.
        K, dist: intrinsic calibration for this camera.
        
    Returns:
        rvec, tvec: rotation and translation from world to camera.
    """
    if world_objpoints is None:
        wp = np.zeros((pattern_size[0]*pattern_size[1],3), np.float32)
        wp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1,2)
        wp *= square_size
        world_objpoints = wp

    for fname in glob.glob(image_glob):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
        if not ret:
            continue
        corners2 = cv2.cornerSubPix(
            gray, corners, (11,11), (-1,-1),
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        ret, rvec, tvec = cv2.solvePnP(world_objpoints, corners2, K, dist)
        if ret:
            proj, _ = cv2.projectPoints(world_objpoints, rvec, tvec, K, dist)
            err = np.linalg.norm(corners2.reshape(-1,2) - proj.reshape(-1,2)) / len(proj)
            print(f"SolvePnP reprojection error: {err*1e3:.2f} mm")
            return rvec, tvec
    raise RuntimeError("Marker not detected in any image")

if __name__ == "__main__":
    # 1) Intrinsics
    _, K1, d1, _, _ = calibrate_intrinsics("calib_images/cam1/*.png")
    _, K2, d2, _, _ = calibrate_intrinsics("calib_images/cam2/*.png")
    
    # 2) Stereo extrinsics
    R, T, E, F = stereo_calibrate(
        "calib_images/cam1/*.png", "calib_images/cam2/*.png",
        pattern_size=(9,6), square_size=0.025,
        K1=K1, d1=d1, K2=K2, d2=d2
    )
    
    # 3) Camera-to-world
    rvec1, tvec1 = calibrate_camera_to_world(
        "marker_images/cam1/*.png", None,
        pattern_size=(9,6), square_size=0.025,
        K=K1, dist=d1
    )
    rvec2, tvec2 = calibrate_camera_to_world(
        "marker_images/cam2/*.png", None,
        pattern_size=(9,6), square_size=0.025,
        K=K2, dist=d2
    )
    
    print("Calibration complete.")


In [ ]:
import cv2
print(cv2.__version__)

# Calibration ChArUco

In [ ]:
import cv2.aruco as aruco
import cv2
import numpy as np
import glob
import os

print("OpenCV version:", cv2.__version__)

def calibrate_intrinsics_charuco(glob_pattern, charuco_board, aruco_dict,
                                 min_markers=20):
    all_corners, all_ids, img_size = [], [], None

    for fname in glob.glob(glob_pattern):
        img = cv2.imread(fname)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img_size is None:
            img_size = img_gray.shape[::-1]

        detector = aruco.CharucoDetector(charuco_board)
        charuco_corners, charuco_ids, marker_corners, marker_ids = detector.detectBoard(img_gray)

        all_corners.append(charuco_corners)
        all_ids.append(charuco_ids)

    # 3. Calibrate
    ret, K, dist, _, _ = aruco.calibrateCameraCharuco(
        charucoCorners=all_corners,
        charucoIds=all_ids,
        board=charuco_board,
        imageSize=img_size,
        cameraMatrix=None,
        distCoeffs=None
    )
    print(f"Charuco intrinsics RMS error: {ret:.4f}")
    return K, dist

def stereo_calibrate_charuco(glob1, glob2, charuco_board, aruco_dict,
                             K1, d1, K2, d2, flags=cv2.CALIB_FIX_INTRINSIC):
    objpoints, imgpts1, imgpts2 = [], [], []
    size = None

    # Pair up images
    for f1, f2 in zip(sorted(glob.glob(glob1)), sorted(glob.glob(glob2))):
        im1, im2 = cv2.imread(f1), cv2.imread(f2)
        g1, g2 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY), cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
        if size is None: size = g1.shape[::-1]

        # Detect & interpolate
        c1, id1, _ = aruco.detectMarkers(g1, aruco_dict)
        c2, id2, _ = aruco.detectMarkers(g2, aruco_dict)
        r1, cc1, cid1 = aruco.interpolateCornersCharuco(c1, id1, g1, charuco_board)
        r2, cc2, cid2 = aruco.interpolateCornersCharuco(c2, id2, g2, charuco_board)
        if r1 < 20 or r2 < 20: continue

        # Build object points (same for both)
        objp = charuco_board.chessboardCorners[cid1.flatten()]
        objpoints.append(objp)
        imgpts1.append(cc1)
        imgpts2.append(cc2)

    # Stereo calibrate
    rms, _, _, _, _, R, T, E, F = cv2.stereoCalibrate(
        objpoints, imgpts1, imgpts2,
        K1, d1, K2, d2, size,
        criteria=(cv2.TERM_CRITERIA_MAX_ITER|cv2.TERM_CRITERIA_EPS,100,1e-5),
        flags=flags
    )
    print(f"Stereo RMS error: {rms:.4f}")
    return R, T


def camera_to_world_charuco(glob_pattern, charuco_board, aruco_dict, K, dist):
    for fname in glob.glob(glob_pattern):
        im = cv2.imread(fname); gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        # 1. Detect markers + ChArUco
        markers, ids, _ = aruco.detectMarkers(gray, aruco_dict)
        ret, cc, cid = aruco.interpolateCornersCharuco(markers, ids, gray, charuco_board)
        if ret < 20: continue

        # 2. Get the corresponding 3D object points
        obj_pts = charuco_board.chessboardCorners[cid.flatten()]

        # 3. SolvePnP
        _, rvec, tvec = cv2.solvePnP(obj_pts, cc, K, dist)
        # 4. (Optional) compute reprojection error
        proj, _ = cv2.projectPoints(obj_pts, rvec, tvec, K, dist)
        err = np.linalg.norm(cc.reshape(-1,2) - proj.reshape(-1,2)) / len(proj)
        print(f"Cam→World PnP error: {err*1000:.2f} mm")
        return rvec, tvec

    raise RuntimeError("ChArUco board not found in any image")


if __name__ == "__main__":

    # Define ChArUco board
    square_size = 0.006
    marker_length = 0.004
    board_size = (10, 10)
    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
    charuco_board = aruco.CharucoBoard(
        size=board_size,
        squareLength=square_size, 
        markerLength=marker_length,
        dictionary=aruco_dict
    )

    # Calibrate instrinsices
    K1, d1 = calibrate_intrinsics_charuco(
        "../camera_calibration/05-08-25/charuco_calib_images/cam0/*.png", charuco_board, aruco_dict
    )

    # save calibration data
    
    print("Calibration complete.")


# Pose Estimation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def back_project(coord, camera_number):
    """
    Back-project a 3D point to 2D pixel coordinates in the image plane.
    
    Args:
        coord: 3D point in world coordinates.
        camera_number: 0 for left camera, 1 for right camera.
        
    Returns:
        pixel_coord: 2D pixel coordinates in the image plane.
    """
    # Camera intrinsics and extrinsics (example values)
    K = np.array([[1000, 0, 320],
                  [0, 1000, 240],
                  [0, 0, 1]])
    R = np.eye(3) if camera_number == 0 else np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    T = np.array([0.5, 0, 0]) if camera_number == 1 else np.array([0, 0, 0])
    
    # Project the point
    coord_homogeneous = np.append(coord, 1) # Convert to homogeneous coordinates
    pixel_coord_homogeneous = K @ (R @ coord_homogeneous + T)
    pixel_coord = pixel_coord_homogeneous[:2] / pixel_coord_homogeneous[2]
    
    return pixel_coord

# 1) Create lookup table for voxel grid
Nx, Ny, Nz = 20, 20, 20
origin = np.array([0, 0, 0])
voxel_size = np.array([0.01, 0.01, 0.01])
lookup_table = np.zeros((Nx, Ny, Nz, 2, 2), dtype=np.float32)
for x in range(Nx):
    for y in range(Ny):
        for z in range(Nz):
            coord = origin + np.array([x, y, z]) * voxel_size
            for camera_number in range(2):
                pixel_coord = back_project(coord, camera_number)
                lookup_table[x, y, z, camera_number] = np.reshape(pixel_coord, 2)

In [ ]:
# 2) Obtain masks for left and right cameras
# TODO: Replace with actual image loading and processing
mask_0 = np.zeros((480, 640), dtype=np.uint8)
mask_1 = np.zeros((480, 640), dtype=np.uint8)

# 3) Voxel carving: keep (x,y,z) if both silhouettes agree
volume = np.zeros((Nx, Ny, Nz), dtype=np.uint8)
for x in range(Nx):
    for y in range(Ny):
        for z in range(Nz):
            pixel_coord_0 = lookup_table[x, y, z, 0]
            pixel_coord_1 = lookup_table[x, y, z, 1]
            if mask_0[pixel_coord_0[0], pixel_coord_0[1]] and \
               mask_1[pixel_coord_1[0], pixel_coord_1[1]]:
                volume[x, y, z] = 1
                
points = np.argwhere(volume == 1)  # Get indices of carved voxels
                

# 4) Compute a simple centerline: for each z-slice, average x & y
centerline = []
for zk in z:
    slice_pts = points[np.isclose(points[:, 2], zk)]
    if len(slice_pts) > 0:
        cx, cy = slice_pts[:,0].mean(), slice_pts[:,1].mean()
        centerline.append([cx, cy, zk])
centerline = np.array(centerline)

# 5) Estimate tangent at the tip (last two points)
v = centerline[-1] - centerline[-2]
theta = np.arccos(v[2] / np.linalg.norm(v))  # bending magnitude
phi   = np.arctan2(v[1], v[0])              # bending plane

print("Estimated bending angle θ (rad):", theta)
print("Estimated bending plane φ (rad):", phi)

# 6) Plot a 2D x–z view of the carved voxels and the centerline
fig, ax = plt.subplots()
ax.scatter(points[:,0], points[:,2], marker='.', label='voxels')
ax.plot(centerline[:,0], centerline[:,2], label='centerline')
ax.set_xlabel('x')
ax.set_ylabel('z')
ax.legend()
plt.show()


# Create Charuco board

In [ ]:
import cv2
import cv2.aruco as aruco
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
import numpy as np

# Parameters for the ChArUco board
SQUARE_SIZE_MM = 6   # Desired square size in mm
MARKER_SIZE_MM = 4   # Desired marker size in mm
SQUARES_X = 10       # Number of squares along x-axis
SQUARES_Y = 10       # Number of squares along y-axis
MARGIN_MM = 10       # Margin around the board

# Create the ArUco dictionary
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

# Define the board size as a tuple (number of squares in x and y directions)
board_size = (SQUARES_X, SQUARES_Y)

# Create the Charuco board using the updated constructor
charuco_board = aruco.CharucoBoard(
    size=board_size,
    squareLength=SQUARE_SIZE_MM / 1000.0,  # in meters
    markerLength=MARKER_SIZE_MM / 1000.0,  # in meters
    dictionary=aruco_dict
)

# Generate the board image using OpenCV
img_size = (SQUARES_X * 300, SQUARES_Y * 300)  # Higher resolution for print quality
img = charuco_board.generateImage(img_size)

# Save the board image as a PNG for reference
cv2.imwrite("charuco_board.png", img)

# Calculate the final PDF size including margins
pdf_width = (SQUARES_X * SQUARE_SIZE_MM) + (2 * MARGIN_MM)  # mm
pdf_height = (SQUARES_Y * SQUARE_SIZE_MM) + (2 * MARGIN_MM)  # mm

# Create a PDF using ReportLab with precise dimensions
pdf_path = "charuco_board_accurate.pdf"
c = canvas.Canvas(pdf_path, pagesize=(pdf_width * mm, pdf_height * mm))

# Convert the OpenCV image to a format suitable for PDF
_, buffer = cv2.imencode(".png", img)
image_data = buffer.tobytes()

# Calculate the exact position for centering the image
x_offset = MARGIN_MM * mm
y_offset = MARGIN_MM * mm
image_width = (SQUARES_X * SQUARE_SIZE_MM) * mm
image_height = (SQUARES_Y * SQUARE_SIZE_MM) * mm

# Draw the ChArUco board image onto the PDF
c.drawImage("charuco_board.png", x_offset, y_offset, image_width, image_height)

# Save the PDF
c.showPage()
c.save()
print(f"Saved accurate PDF: {pdf_path}")


# Pixel Color Classifier

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

class PixelColorClassifier:
    def __init__(self, model_type='logistic', color_space='HSV'):
        """
        Parameters:
            model_type: 'logistic' or 'naive_bayes'
            color_space: 'HSV', 'RGB', or 'BGR'
        """
        if model_type == 'logistic':
            self.model = LogisticRegression(max_iter=200)
        elif model_type == 'naive_bayes':
            self.model = GaussianNB()
        else:
            raise ValueError("Unsupported model_type. Choose 'logistic' or 'naive_bayes'.")
        
        self.model_type = model_type
        self.color_space = color_space
        self.trained = False

    def _convert_color(self, image_bgr):
        import cv2
        if self.color_space == 'HSV':
            return cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
        elif self.color_space == 'RGB':
            return cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        elif self.color_space == 'BGR':
            return image_bgr
        else:
            raise ValueError("Unsupported color space")

    def fit(self, image_bgr, fg_mask, bg_mask, max_samples=1000):
        """
        Train the model using foreground and background binary masks.

        Parameters:
            image_bgr: Input image (BGR format)
            fg_mask: Binary mask where foreground pixels are 255
            bg_mask: Binary mask where background pixels are 255
        """
        img_color = self._convert_color(image_bgr)
        H, W, _ = img_color.shape
        flat_img = img_color.reshape(-1, 3)

        fg_indices = np.where(fg_mask.flatten() > 0)[0]
        bg_indices = np.where(bg_mask.flatten() > 0)[0]

        n_fg = min(max_samples, len(fg_indices))
        n_bg = min(max_samples, len(bg_indices))

        fg_sample = np.random.choice(fg_indices, n_fg, replace=False)
        bg_sample = np.random.choice(bg_indices, n_bg, replace=False)

        X = np.vstack((flat_img[fg_sample], flat_img[bg_sample]))
        y = np.hstack((np.ones(n_fg), np.zeros(n_bg)))

        self.model.fit(X, y)
        self.trained = True

    def predict_proba(self, image_bgr):
        """
        Predict foreground probability for each pixel.

        Parameters:
            image_bgr: Input image (BGR format)

        Returns:
            prob_map: 2D numpy array of foreground probabilities
        """
        if not self.trained:
            raise RuntimeError("PixelColorClassifier must be trained with `fit()` before prediction.")

        img_color = self._convert_color(image_bgr)
        H, W, _ = img_color.shape
        flat_img = img_color.reshape(-1, 3)

        proba = self.model.predict_proba(flat_img)[:, 1]
        return proba.reshape(H, W)


# Collect Intrinsic Calib Images

In [ ]:
import cv2
import os
import subprocess

def configure_camera(devices):
    for device in devices:

        print(f"Configuring camera on {device}...")

        # Build the commands to configure the camera
        commands = [
            f"v4l2-ctl -d {device} -c focus_automatic_continuous=0",
            f"v4l2-ctl -d {device} -c auto_exposure=3",
            f"v4l2-ctl -d {device} -c focus_absolute=35",
            # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
            # f"v4l2-ctl -d {device} -c gain=0",
            # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
            # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
            # f"v4l2-ctl -d {device} -c brightness=128",
            # f"v4l2-ctl -d {device} -c contrast=128",
            # f"v4l2-ctl -d {device} -c saturation=128",
        ]

        for command in commands:
            subprocess.run(command, shell=True, check=True)

        print("Camera configuration complete!")

# Camera settings
camera_id = [0]
devices = [f"/dev/video{i}" for i in camera_id] 
configure_camera(devices)
output_dir = "../camera_calibration/05-08-25/charuco_calib_imagescalib_images/cam1"  # Directory to save images
os.makedirs(output_dir, exist_ok=True)

# Initialize webcam
cap = cv2.VideoCapture(camera_id[0], cv2.CAP_V4L2)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

print("Press SPACE to capture image, ESC to exit.")

image_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # Display the live feed
    cv2.imshow("Camera Feed - Press SPACE to capture, ESC to exit", frame)

    # Keyboard input
    key = cv2.waitKey(1) & 0xFF

    # Save image on SPACE key press
    if key == ord(' '):
        # Construct image filename
        image_path = os.path.join(output_dir, f"img_{image_count:03d}.png")
        # Save the frame as PNG
        cv2.imwrite(image_path, frame)
        print(f"Captured: {image_path}")
        image_count += 1

    # Exit on ESC key press
    elif key == 27:
        print("Exiting...")
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()


# Collect Extrinsic Calib Images

In [ ]:
import cv2
import os
import subprocess

def configure_camera(devices):
    for device in devices:

        print(f"Configuring camera on {device}...")

        # Build the commands to configure the camera
        commands = [
            f"v4l2-ctl -d {device} -c focus_automatic_continuous=0",
            f"v4l2-ctl -d {device} -c auto_exposure=3",
            f"v4l2-ctl -d {device} -c focus_absolute=35",
            # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
            # f"v4l2-ctl -d {device} -c gain=0",
            # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
            # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
            # f"v4l2-ctl -d {device} -c brightness=128",
            # f"v4l2-ctl -d {device} -c contrast=128",
            # f"v4l2-ctl -d {device} -c saturation=128",
        ]

        for command in commands:
            subprocess.run(command, shell=True, check=True)

        print("Camera configuration complete!")

# Camera settings
camera_id = 1
devices = [f"/dev/video0"] 
configure_camera(devices)
output_dir = f"../camera_calibration/05-08-25/extrinsic_calib_images/cam{camera_id}"  # Directory to save images
os.makedirs(output_dir, exist_ok=True)

# Initialize webcam
cap = cv2.VideoCapture(0, cv2.CAP_V4L2)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

print("Press SPACE to capture image, ESC to exit.")

image_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # Display the live feed
    cv2.imshow("Camera Feed - Press SPACE to capture, ESC to exit", frame)

    # Keyboard input
    key = cv2.waitKey(1) & 0xFF

    # Save image on SPACE key press
    if key == ord(' '):
        # Construct image filename
        image_path = os.path.join(output_dir, f"img_{image_count:03d}.png")
        # Save the frame as PNG
        cv2.imwrite(image_path, frame)
        print(f"Captured: {image_path}")
        image_count += 1

    # Exit on ESC key press
    elif key == 27:
        print("Exiting...")
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()

# Find correct focus distance   

In [ ]:
import cv2
import subprocess

def set_focus(value):
    command = f"v4l2-ctl -d /dev/video0 -c focus_absolute={value}"
    subprocess.run(command, shell=True)

cap = cv2.VideoCapture(0, cv2.CAP_V4L2)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)  # Disable autofocus

print("Use the UP and DOWN arrow keys to adjust focus. Press ESC to exit.")
focus_value = 120  # Start with a mid-range value
set_focus(focus_value)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read from camera.")
        break

    # Display current focus value on the frame
    cv2.putText(frame, f"Focus: {focus_value}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Adjust Focus - Use UP/DOWN keys", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC key to exit
        break
    elif key == ord('w'):  # Increase focus
        focus_value = min(focus_value + 5, 250)
        set_focus(focus_value)
        print(f"Focus set to: {focus_value}")
    elif key == ord('s'):  # Decrease focus
        focus_value = max(focus_value - 5, 0)
        set_focus(focus_value)
        print(f"Focus set to: {focus_value}")

cap.release()
cv2.destroyAllWindows()


# Capture simultaneous images from two webcams

In [ ]:
import cv2
from datetime import datetime
import os

# Directory to save images
output_dir = "../stereo_images"
os.makedirs(output_dir, exist_ok=True)

# Open both webcams
cap1 = cv2.VideoCapture(0, cv2.CAP_V4L2)  # First webcam at /dev/video0, top cam
cap2 = cv2.VideoCapture(2, cv2.CAP_V4L2)  # Second webcam at /dev/video2, side cam

# Set resolution and frame rate for both cameras
# cap1.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
# cap1.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
# cap1.set(cv2.CAP_PROP_FPS, 30)

# cap2.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
# cap2.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
# cap2.set(cv2.CAP_PROP_FPS, 30)

if not cap1.isOpened() or not cap2.isOpened():
    print("Error: One or both cameras could not be opened.")
    exit()

print("Press SPACE to capture simultaneous images, ESC to exit.")

frame_count = 0

while True:
    # Read frames from both cameras
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1 or not ret2:
        print("Error: One or both frames could not be read.")
        break

    # Display both camera feeds with timestamps
    # timestamp = datetime.now().strftime("%H:%M:%S.%f")
    # cv2.putText(frame1, f"Cam1 - {timestamp}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # cv2.putText(frame2, f"Cam2 - {timestamp}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Combine and display both frames
    combined = cv2.hconcat([frame1, frame2])
    cv2.imshow("Camera 0 (top) + Camera 2 (side)", combined)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord(' '):  # Space key to capture images
        # Save images with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        img1_path = f"{output_dir}/cam0_{frame_count}.png"
        img2_path = f"{output_dir}/cam1_{frame_count}.png"
        cv2.imwrite(img1_path, frame1)
        cv2.imwrite(img2_path, frame2)
        print(f"Captured images:\n - {img1_path}\n - {img2_path}")
        frame_count += 1

# Release both cameras and close windows
cap1.release()
cap2.release()
cv2.destroyAllWindows()


# Camera-Camera Calibration

In [ ]:
import cv2
import cv2.aruco as aruco
import numpy as np
import glob
import pickle

def detect_charuco_corners(image_path, board, detector):
    """Detects Charuco corners using OpenCV 4.11+ functions."""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect Aruco markers and Charuco corners
    charuco_corners, charuco_ids, _, _ = detector.detectBoard(gray)

    if charuco_ids is not None and len(charuco_ids) > 4:
        return charuco_corners, charuco_ids
    return None, None

def calculate_relative_transform(R1, T1, R2, T2):
    """Calculate the relative rotation and translation between two cameras.
    Returns R21 and T21, the rotation and translation of camera 2 in camera 1's frame."""
    R21 = np.dot(R2, R1.T)
    T21 = T2 - np.dot(R21, T1)
    return R21, T21

def get_extrinsics(image_path, K, dist, board, detector):
    """Compute extrinsics for a single image."""
    charuco_corners, charuco_ids = detect_charuco_corners(image_path, board, detector)
    if charuco_corners is not None:
        obj_points = board.getChessboardCorners()[charuco_ids.flatten()]
        ret, rvec, tvec = cv2.solvePnP(obj_points, charuco_corners, K, dist)
        if ret:
            R, _ = cv2.Rodrigues(rvec)
            return R, tvec
    return None, None

# Load intrinsic parameters
calib_data_file = "../camera_calibration/05-08-25/camera_calib_data.pkl"
with open(calib_data_file, 'rb') as f:
    calib_data = pickle.load(f)

K1 = calib_data["cam0"]["intrinsics"]["K"]
dist1 = calib_data["cam0"]["intrinsics"]["d"]
K2 = calib_data["cam1"]["intrinsics"]["K"]
dist2 = calib_data["cam1"]["intrinsics"]["d"]

# Create Aruco dictionary and Charuco board
# Parameters for the ChArUco board
SQUARE_SIZE_MM = 6   # Desired square size in mm
MARKER_SIZE_MM = 4   # Desired marker size in mm
SQUARES_X = 10       # Number of squares along x-axis
SQUARES_Y = 10       # Number of squares along y-axis

# Create the ArUco dictionary
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

# Define the board size as a tuple (number of squares in x and y directions)
board_size = (SQUARES_X, SQUARES_Y)

# Create the Charuco board using the updated constructor
charuco_board = aruco.CharucoBoard(
    size=board_size,
    squareLength=SQUARE_SIZE_MM / 1000.0,  # in meters
    markerLength=MARKER_SIZE_MM / 1000.0,  # in meters
    dictionary=aruco_dict
)

# Create the Aruco and Charuco detector objects (new in OpenCV 4.11+)
aruco_detector = aruco.ArucoDetector(aruco_dict)
charuco_detector = aruco.CharucoDetector(charuco_board)

# Paths to synchronized images
cam1_images = sorted(glob.glob("../stereo_images/cam0_*.png"))
cam2_images = sorted(glob.glob("../stereo_images/cam1_*.png"))

# Arrays to store transformations
relative_rotations = []
relative_translations = []

for img1_path, img2_path in zip(cam1_images, cam2_images):
    # Get extrinsics for both cameras using updated functions
    R1, T1 = get_extrinsics(img1_path, K1, dist1, charuco_board, charuco_detector)
    R2, T2 = get_extrinsics(img2_path, K2, dist2, charuco_board, charuco_detector)

    if R1 is not None and R2 is not None:
        # Calculate the relative transformation between cameras
        R21, T21 = calculate_relative_transform(R1, T1, R2, T2)
        relative_rotations.append(R21)
        relative_translations.append(T21)

# Average the transformations
R_avg = sum(relative_rotations) / len(relative_rotations)
T_avg = sum(relative_translations) / len(relative_translations)

# Print the averaged relative transformation
print("Averaged Relative Rotation (R21):\n", R_avg)
print("Averaged Relative Translation (T21):\n", T_avg)

# Save the relative transformation
np.savez("relative_transformation.npz", R21=R_avg, T21=T_avg)
print("Relative transformation saved to 'relative_transformation.npz'")


Averaged Relative Rotation (R21):
 [[ 0.00441847  0.02945237 -0.99954709]
 [-0.05544742  0.99801003  0.0291781 ]
 [ 0.9984277   0.05528752  0.0060255 ]]
Averaged Relative Translation (T21):
 [[0.17086164]
 [0.00920667]
 [0.22905064]]
Relative transformation saved to 'relative_transformation.npz'


In [3]:
import numpy as np

def rotation_angle(R):
    """Calculate the rotation angle from the rotation matrix."""
    trace = np.trace(R)
    theta = np.arccos((trace - 1) / 2)
    return np.degrees(theta)

def rotation_axis(R):
    """Calculate the axis of rotation from the rotation matrix."""
    angle = rotation_angle(R)
    axis = np.array([
        R[2, 1] - R[1, 2],
        R[0, 2] - R[2, 0],
        R[1, 0] - R[0, 1]
    ])
    axis = axis / np.linalg.norm(axis)
    return axis, angle

# Relative Rotation Matrix (from your calibration)
R21 = np.array([
    [0.0044, 0.0295, -0.9995],
    [-0.0554, 0.9980, 0.0292],
    [0.9984, 0.0553, 0.0060]
])

# Relative Translation Vector (from your calibration)
T21 = np.array([0.1709, 0.0092, 0.2291])

# Calculate the rotation angle and axis
axis, angle = rotation_axis(R21)

print(f"Rotation Angle: {angle:.2f} degrees")
print(f"Rotation Axis: {axis}")

# Interpretation of Translation
print(f"Relative Translation (T21): {T21}")

# Flip the interpretation to see if the reverse makes sense
T12 = -np.dot(R21.T, T21)
print(f"Flipped Translation (T12): {T12}")


Rotation Angle: 89.76 degrees
Rotation Axis: [ 0.01305083 -0.99901324 -0.04245269]
Relative Translation (T21): [0.1709 0.0092 0.2291]
Flipped Translation (T12): [-0.22897572 -0.02689238  0.16917131]


In [ ]:
import cv2
import cv2.aruco as aruco
import numpy as np
import pickle
import glob

def project_points(points_3d, R, T, K, dist):
    """Projects 3D points to 2D image plane using extrinsic and intrinsic parameters."""
    projected_points, _ = cv2.projectPoints(points_3d, cv2.Rodrigues(R)[0], T, K, dist)
    return projected_points.squeeze()

def draw_points(image, points, color, label):
    """Draws points on the image."""
    for point in points:
        cv2.circle(image, tuple(point.astype(int)), 5, color, -1)
    # Label the points for clarity
    cv2.putText(image, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

def visualize_relative_transformation(img1_path, img2_path, R1, T1, R2, T2, K1, dist1, K2, dist2, R21, T21):
    """Visualize relative transformation between two cameras."""
    # Load the images
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    # Define a set of 3D points around the origin (world frame)
    points_3d = np.array([
        [0, 0, 0],       # Origin
        [0.05, 0, 0],    # X-axis point
        [0, 0.05, 0],    # Y-axis point
        [0, 0, 0.05],    # Z-axis point
        [0.05, 0.05, 0], # XY-plane point
        [0.05, 0, 0.05], # XZ-plane point
        [0, 0.05, 0.05], # YZ-plane point
    ], dtype=np.float32)

    # Project points from world frame to camera 0 frame
    projected_cam1 = project_points(points_3d, R1, T1, K1, dist1)

    # Transform the points to the second camera using the relative transformation
    transformed_points = (R21 @ points_3d.T).T + T21.T
    projected_cam2 = project_points(transformed_points, R2, T2, K2, dist2)

    # Draw both detected and projected points on both images
    draw_points(img1, projected_cam1, (0, 255, 0), "Camera 0: Projected")  # Green: Projected on Cam0
    draw_points(img2, projected_cam2, (255, 0, 0), "Camera 1: Projected")  # Blue: Projected on Cam1

    # Combine the images for side-by-side visualization
    combined = cv2.hconcat([img1, img2])
    cv2.imshow("Calibration Verification: Camera 0 (Left) | Camera 1 (Right)", combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Load the relative transformation between cameras
rel_data = np.load("relative_transformation.npz")
R21 = rel_data['R21']
T21 = rel_data['T21']

# Load intrinsic parameters
calib_data_file = "../camera_calibration/05-08-25/camera_calib_data.pkl"
with open(calib_data_file, 'rb') as f:
    calib_data = pickle.load(f)

K1 = calib_data["cam0"]["intrinsics"]["K"]
dist1 = calib_data["cam0"]["intrinsics"]["d"]
K2 = calib_data["cam1"]["intrinsics"]["K"]
dist2 = calib_data["cam1"]["intrinsics"]["d"]

# Paths to synchronized images
cam1_images = sorted(glob.glob("../stereo_images/cam0_*.png"))
cam2_images = sorted(glob.glob("../stereo_images/cam1_*.png"))

# Load the first image pair for visualization
img1_path = cam1_images[0]
img2_path = cam2_images[0]

# Load the pre-computed extrinsics for both cameras
R1, T1 = get_extrinsics(img1_path, K1, dist1, charuco_board, charuco_detector)
R2, T2 = get_extrinsics(img2_path, K2, dist2, charuco_board, charuco_detector)

# Visualize the relative transformation using the first image pair
visualize_relative_transformation(img1_path, img2_path, R1, T1, R2, T2, K1, dist1, K2, dist2, R21, T21)


## Camera-World Calibration with Aruco Marker

In [ ]:
# Capture image for camera-world calibration
import cv2

camera_id = 0 # use top camera
devices = [f"/dev/video0"]
configure_camera(devices)
output_dir = "../camera_calibration/05-08-25/camera_world_calib_images/cam0"  # Directory to save images

In [ ]:
# Calculate camera to world frame transform using aruco marker


